# Example 1: Single Transmon

In [ ]:
%load_ext autoreload
%autoreload 2
import time

In [ ]:
import json

import design as d

with open("design_variables.json") as in_file:
    dv = json.load(in_file)
import mini_studies as ms
import optimization_targets as ot
import plot_settings as ps
import target_parameters as tp

import qdesignoptimizer.utils.utils_design_variables as u
from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qdesignoptimizer.utils.utils_design import create_chip_base

## Design assembly

In [ ]:
CHIP_NAME = "transmon_chip"
OPEN_GUI = True
CHIP_TYPE = {"size_x": "10mm", "size_y": "10mm", "size_z": "-300um"}
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=CHIP_TYPE, open_gui=OPEN_GUI
)

u.add_design_variables_to_design(design, dv)

In [ ]:
def render_qiskit_metal_design(design, gui):
    d.add_branch(design, 0, gui)
    d.add_branch(design, 1, gui)
    d.add_branch(design, 2, gui)
    d.add_branch(design, 3, gui)
    d.add_branch(design, 4, gui)

    d.add_route_interconnects(design, 0, gui)
    d.add_route_interconnects(design, 1, gui)
    d.add_route_interconnects(design, 2, gui)
    d.add_route_interconnects(design, 3, gui)

    d.add_launch_pads(design, gui)


render_qiskit_metal_design(design, gui)

## MiniStudy: qubit and resonator

In [ ]:
MINI_STUDY_BRANCH = 0
MINI_STUDY = ms.get_mini_study_qb_res(branch=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)
opt_targets = ot.get_opt_targets_qb_res(branch=MINI_STUDY_BRANCH)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, tp.TARGET_PARAMS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters=False,
    plot_settings=ps.PLOT_SETTINGS,
    plot_branches_separately=False,
)


nbr_runs = 8
nbr_passes = 5

delta_f = 0.001
for i in range(nbr_runs):
    design_analysis.update_nbr_passes(nbr_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

## MiniStudy: qubit decay in charge line
The T1 of the qubit is limited by its decay into the charge line.
This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
MINI_STUDY_BRANCH = 0
MINI_STUDY = ms.get_mini_study_qb_charge_line(branch=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_targets_qb_charge_line(branch=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
%matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, tp.TARGET_PARAMS_CHARGE_LINE_DECAY
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_parameters=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
    plot_branches_separately=False,
)


nbr_runs = 5
nbr_passes_cap = 8

for i in range(nbr_runs):
    design_analysis.update_nbr_passes_capacitance_ministudies(nbr_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

In [ ]:
from qdesignoptimizer.utils.utils import close_ansys

close_ansys()